# 🤵 Примеры работы классических *unsupervised* методов обнаружения аномалий

### 🌐 Установка [pyod](https://github.com/yzhao062/pyod)

In [ ]:
!pip3 install pyod

     |████████████████████████████████| 102kB 9.9MB/s 
  Created wheel for pyod: filename=pyod-0.8.7-cp37-none-any.whl size=115979 sha256=0f688a4722c36d3146f7289c88a8d178c37d6b68b5c9875e8c8be2b7ca45888d
  Stored in directory: /root/.cache/pip/wheels/b7/8a/c9/5fe3f32692ad445fe573cf5b953d7a0d55dc1f7c8e229ebda0
Successfully built pyod


## ⚙️ Загрузка данных

In [ ]:
from sklearn.datasets import fetch_20newsgroups


c = 0.1  # отношение количества аномальных экземпляров к нормальным

normal_data = fetch_20newsgroups(subset='all', categories=['sci.electronics'],
                               shuffle=True, random_state=123, 
                               remove=['headers', 'footers'], return_X_y=True)[0]
anomal_data = fetch_20newsgroups(subset='all', categories=['talk.politics.mideast'],
                               shuffle=True, random_state=123,
                               remove=['headers', 'footers'],
                               return_X_y=True)[0][:int(c * len(normal_data)) + 1]

print("Количество нормальных экземпляров = {}".format(len(normal_data)))
print("Количество аномальных экземпляров = {}".format(len(anomal_data)))

Количество нормальных экземпляров = 984
Количество аномальных экземпляров = 99


## 💅 Предобработка данных

In [ ]:
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from gensim.utils import tokenize
import nltk; nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
# from nltk.util import ngrams


def strip_html_tags(text):
    """Удаление html tags из текста."""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def preprocess_text(text):
    text = strip_html_tags(text)  # удаление html tags
    text = strip_non_alphanum(text) # заменили все небуквенные символы на пробел
    text = strip_numeric(text) # удалили все цифры
    text = remove_stopwords(text) # удалили все стоп-слова
    # text = strip_short(text, minsize=2) # удалили короткие слова
    word_list = list(tokenize(text, deacc=True, to_lower=True)) # токенизация, deacc - избавляет от ударений
    word_list = [WordNetLemmatizer().lemmatize(word) for word in word_list] # лемматизация
    return ' '.join(word for word in word_list)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
normal_data = [preprocess_text(text) for text in normal_data]
anomal_data = [preprocess_text(text) for text in anomal_data]
all_data = normal_data + anomal_data

## 🎰 TF-IDF векторизация

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
all_data_tf = vectorizer.fit_transform(all_data).toarray()

print(all_data_tf.shape)

(1083, 13828)


## 📝 Universal Sentence Encoder (USE)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
hub_layer = hub.KerasLayer(
    'https://tfhub.dev/google/universal-sentence-encoder/4',
    input_shape=[], 
    dtype=tf.string,
    trainable=False)

## 🤹‍♂️ Формирование выборок



In [ ]:
from sklearn.utils import shuffle
import numpy as np

x = all_data_tf
# x_use = (hub_layer(all_data)).numpy()
y = np.array([False] * len(normal_data) + [True] * len(anomal_data))

all_data, x, y = shuffle(all_data, x, y, random_state=123)
print("Всего экземпляров = {}".format(len(all_data)))
print("(Кол-во текстов, число признаков текста) = {}".format(x.shape))
print("Кол-во меток = {}".format(len(y)))
print("Кол-во нормальных экземпляров = {}".format(len(normal_data)))
print("Кол-во аномальных экземпляров = {}".format(len(anomal_data)))

Всего экземпляров = 1083
(Кол-во текстов, число признаков текста) = (1083, 13828)
Кол-во меток = 1083
Кол-во нормальных экземпляров = 984
Кол-во аномальных экземпляров = 99


## 1️⃣ 🌲 Isolation Forest (IF, Изолирующий лес)
У меня на практике 5000 деревьев дают такой же результат, что и 15000.

### 🅰️ Метод из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html)

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score

# Задаем модель
clf = IsolationForest(
    contamination=0.1, # отношение аномалий к выборке
    n_estimators=5000, # кол-во деревьев в ансамбле
    max_samples=0.7,
    max_features=0.7,
    random_state=123, 
    bootstrap=True,
    n_jobs=-1, # задействем все процессы в распоряжении
)

# Тренировка модели
clf.fit(x)

# Предсказание модели
y_predict_iforest = clf.predict(x)

# Переименовываем метки в соответствии с нашими
y_predict_iforest[y_predict_iforest == 1] = False
y_predict_iforest[y_predict_iforest == -1] = True

# Считаем метрику ROC AUC
auc_iforest = roc_auc_score(y, y_predict_iforest)
print("auc_iforest = ", auc_iforest)

KeyboardInterrupt: ignored

### 🅱️ Метод из [pyod](https://github.com/yzhao062/pyod)
Ответы показывает такие же, что и в sclearn, так как интерпретация на основе их реализации.

####  🎰  TF-IDF

In [ ]:
### TF-IDF ###

import pyod
from pyod.models import iforest
from sklearn.metrics import roc_auc_score

# Задаем модель
iforest_clf = iforest.IForest(
    contamination=0.1,
    n_estimators=5000,
    max_samples=1.0,
    bootstrap=True,
    random_state=123,
    n_jobs=-1,
)

# Тренировка и предсказание модели
# y_predict_iforest = iforest_clf.fit_predict(x)

# Считаем метрику ROC AUC
# auc_iforest = roc_auc_score(y, y_predict_iforest)
auc_iforest = iforest_clf.fit_predict_score(x, y)
print("auc_iforest = ", auc_iforest)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


#### 📝 Universal Sentence Encoder

In [ ]:
### Universal Sentence Encoder ###

import pyod
from pyod.models import iforest
from sklearn.metrics import roc_auc_score

# Задаем модель
iforest_clf = iforest.IForest(
    contamination=0.1,
    n_estimators=5000,
    max_samples=1.0,
    bootstrap=True,
    random_state=123,
    n_jobs=-1,
)

# Тренировка и предсказание модели
y_predict_iforest = iforest_clf.fit_predict(x_use)

# Считаем метрику ROC AUC
auc_iforest = roc_auc_score(y, y_predict_iforest)
print("auc_iforest = ", auc_iforest)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function fit_predict is deprecated
  warnings.warn(msg, category=FutureWarning)


auc_iforest =  0.5224347129834934


## 2️⃣ 🧮 Local Outlier Factor (LOF, Локальный уровень выброса)

###  🎰  TF-IDF

In [ ]:
import pyod
from pyod.models import lof
from sklearn.metrics import roc_auc_score


# Создание модели
lof_clf = lof.LOF(
    contamination=0.1,
    n_neighbors=5,
    metric='canberra',
    n_jobs=-1,
)

# Тренировка и предсказание модели
# y_predict_lof = lof_clf.fit_predict(x)
# auc_lof = roc_auc_score(y, y_predict_lof)

auc_lof = lof_clf.fit_predict_score(x, y)

# Считаем метрику ROC AUC
print("auc_lof = ", auc_lof)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.7647511702389751
auc_lof =  0.7647511702389751


### 📝 Universal Sentence Encoder

In [ ]:
import pyod
from pyod.models import lof
from sklearn.metrics import roc_auc_score


# Создание модели
lof_clf = lof.LOF(
    contamination=0.1,
    n_neighbors=13,
    metric='canberra',
    n_jobs=-1,
)

# Тренировка и предсказание модели
y_predict_lof = lof_clf.fit_predict(x_use)

# Считаем метрику ROC AUC
auc_lof = roc_auc_score(y, y_predict_lof)
print("auc_lof = ", auc_lof)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function fit_predict is deprecated
  warnings.warn(msg, category=FutureWarning)


auc_lof =  0.5335519832471052


## 3️⃣👬🏻 k-nearest neighbors (kNN, Метод k-ближайших соседей)

###  🎰  TF-IDF

In [ ]:
import pyod
from pyod.models import knn
from sklearn.metrics import roc_auc_score

# Создание модели
knn_clf = knn.KNN(
    contamination=0.1,
    n_neighbors=3,
    method='largest',
    metric='canberra',
    n_jobs=-1,
)

# Тренировка и предсказание модели
# y_predict_knn = knn_clf.fit_predict(x)

# Считаем метрику ROC AUC
auc_knn = knn_clf.fit_predict_score(x, y)
# auc_knn = roc_auc_score(y, y_predict_knn)
print("auc_knn = ", auc_knn)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function fit_predict_score is deprecated
  warnings.warn(msg, category=FutureWarning)


roc_auc_score: 0.7340272645150694
auc_knn =  0.7340272645150694


### 📝 Universal Sentence Encoder

In [ ]:
import pyod
from pyod.models import knn
from sklearn.metrics import roc_auc_score

# Создание модели
knn_clf = knn.KNN(
    contamination=0.1,
    n_neighbors=120,
    method='largest',
    metric='l1',
    n_jobs=-1,
)

# Тренировка и предсказание модели
y_predict_knn = knn_clf.fit_predict(x_use)

# Считаем метрику ROC AUC
auc_knn = roc_auc_score(y, y_predict_knn)
print("auc_knn = ", auc_knn)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function fit_predict is deprecated
  warnings.warn(msg, category=FutureWarning)


auc_knn =  0.5502278886425228


## 4️⃣📊 [Copula-Based Outlier Detection](http://www.andrew.cmu.edu/user/yuezhao2/papers/20-icdm-copod.pdf) (COPOD)
[Что такое копула?](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BF%D1%83%D0%BB%D0%B0)

###  🎰  TF-IDF

In [ ]:
import pyod
from pyod.models import copod
from sklearn.metrics import roc_auc_score

# Создание модели
copod_clf = copod.COPOD(contamination=0.1)

# Тренировка и предсказание модели
# y_predict_copod = copod_clf.fit_predict(x)

# Считаем метрику ROC AUC
auc_copod = copod_clf.fit_predict_score(x, y)
# auc_copod = roc_auc_score(y, y_predict_copod)
print("auc_copod = ", auc_copod)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


auc_copod =  0.7503079576250308


### 📝 Universal Sentence Encoder

In [ ]:
import pyod
from pyod.models import copod
from sklearn.metrics import roc_auc_score

# Создание модели
copod_clf = copod.COPOD(contamination=0.1)

# Тренировка и предсказание модели
y_predict_copod = copod_clf.fit_predict(x_use)

# Считаем метрику ROC AUC
auc_copod = roc_auc_score(y, y_predict_copod)
print("auc_copod = ", auc_copod)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function fit_predict is deprecated
  warnings.warn(msg, category=FutureWarning)


auc_copod =  0.5168760778516875


## 5️⃣⌛️ AE (Автокодировщик)

In [ ]:
import pyod
from pyod.models import auto_encoder
from sklearn.metrics import roc_auc_score

# Создание модели
ae_clf = auto_encoder.AutoEncoder(
    hidden_neurons=[1024, 512, 256, 128, 256, 512, 1024],
    hidden_activation='relu',
    output_activation='sigmoid',
    optimizer='adam',
    epochs=50,
    batch_size=256,
    dropout_rate=0.3,
    l2_regularizer=0.1,
    validation_size=0.1,
    preprocessing=True,
    verbose=1,
    random_state=123,
    contamination=0.1
)

# Считаем метрику ROC AUC
ae_clf.fit(x)
y_predict_ae = ae_clf.decision_function(x)
auc_ae = roc_auc_score(y, y_predict_ae)
print("auc_ae = ", auc_ae)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 13828)             191227412 
_________________________________________________________________
dropout_27 (Dropout)         (None, 13828)             0         
_________________________________________________________________
dense_44 (Dense)             (None, 13828)             191227412 
_________________________________________________________________
dropout_28 (Dropout)         (None, 13828)             0         
_________________________________________________________________
dense_45 (Dense)             (None, 1024)              14160896  
_________________________________________________________________
dropout_29 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_46 (Dense)             (None, 512)              

## 6️⃣⏲ VAE (Вариационный автокодировщик)

In [ ]:
import pyod
from pyod.models import vae
from sklearn.metrics import roc_auc_score

# Создание модели
vae_clf = vae.VAE(
    contamination=0.1,
    encoder_neurons=[1024, 512, 256],
    decoder_neurons=[256, 512, 1024],
    latent_dim=5,
    hidden_activation='relu',
    output_activation='sigmoid',
    optimizer='adam',
    epochs=50,
    batch_size=256,
    dropout_rate=0.3,
    l2_regularizer=0.1,
    validation_size=0.1,
    preprocessing=True,
    verbose=1,
    random_state=123,
)

# Считаем метрику ROC AUC
vae_clf.fit(x)
y_predict_vae = vae_clf.decision_function(x)
auc_vae = roc_auc_score(y, y_predict_vae)
print("auc_vae = ", auc_vae)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 13828)]      0                                            
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 13828)        191227412   input_3[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 1024)         14160896    dense_11[0][0]                   
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 1024)         0           dense_12[0][0]                   
____________________________________________________________________________________________